In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_together import Together
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

In [4]:
def get_pdf_text(pdf_docs):
    loader = PyPDFLoader(file_path=pdf_docs)
    pages = loader.load()
    return pages

In [5]:
def get_text_chunks(pages):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500, chunk_overlap = 200, 
        length_function=len, separators=['\n\n', '\n', ' ', '']
    )
    chunks = text_splitter.split_documents(pages)
    
    return chunks

In [6]:
def get_embeddings():
    embeddings = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY, 
                                              model="models/embedding-001")
    
    return embeddings

In [7]:
def get_vectorstore(chunks, embeddings):
    vectorstore = FAISS.from_documents(documents=chunks, 
                                       embedding=embeddings)
    
    return vectorstore

In [10]:
def get_conversation_chain(vectorstore):
    
    llm = Together(model="mistralai/Mistral-7B-Instruct-v0.3", 
                   api_key=TOGETHER_API_KEY, max_tokens=512)
    memory = ConversationBufferMemory(memory_key= "chat_history", 
                                      return_messages=True)
    conversation = ConversationalRetrievalChain.from_llm(llm=llm, 
                                        retriever=vectorstore.as_retriever(), memory=memory)
    
    return conversation

In [11]:
pages = get_pdf_text(pdf_docs="networking-interview-questions-and-answers-pdf.pdf")
chunks = get_text_chunks(pages=pages)
embeddings = get_embeddings()
vectorstore = get_vectorstore(chunks=chunks, embeddings=embeddings)
conversation = get_conversation_chain(vectorstore)

In [13]:
result = conversation("How can Switched Virtual Network be established?")['answer']

In [14]:
print(result)


1. The computer sends a connection request to the switch to which it is attached.
2. Software in the switch finds a network path to the destination.
3. The connection is established by the destination.
4. A message is sent back to the originating computer to indicate the SVC is ready.
5. If any switch or the destination computer does not agree to set up the VC, an error message is sent back and the SVC is not established.
